In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

In [ ]:
from IPython.display import clear_output
# !pip install wandb
clear_output()

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 121754 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3+git20210102-6ubuntu0.2_amd64.deb ...
Unpacking w3m (0.5.3+git20210102-6ubuntu0.2) ...
Setting up w3m (0.5.3+git20210102-6ubuntu0.2) ...
Processing triggers for man-db (2.10.2-1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


In [ ]:
# !pip uninstall contractions
# clear_output()

In [ ]:
import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
# import wandb
import os
import re

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata

# nlp = spacy.load('en', parse = False, tag=False, entity=False)
tokenizer = ToktokTokenizer()
# stopword_list = nltk.corpus.stopwords.words('english')
# stopword_list.remove('no')
# stopword_list.remove('not')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

ImportError: cannot import name 'CONTRACTION_MAP' from 'contractions' (/usr/local/lib/python3.10/dist-packages/contractions/__init__.py)

In [ ]:
%cd /content/
!wget https://raw.githubusercontent.com/Puspita-111/Suven-consultants-and-technology/main/contractions.py
!wget https://raw.githubusercontent.com/Puspita-111/Suven-consultants-and-technology/main/text_normalizer.py

/content


In [ ]:
!ls

contractions.py  drive	sample_data  text_normalizer.py


In [ ]:
!mv contractions.py /content/

In [ ]:
!mv /text_normalizer.py /content/

--2024-02-24 10:38:53--  https://raw.githubusercontent.com/Puspita-111/Suven-consultants-and-technology/main/contractions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3110 (3.0K) [text/plain]
Saving to: ‘contractions.py.4’

contractions.py.4   100%[===================>]   3.04K  --.-KB/s    in 0s      

2024-02-24 10:38:53 (51.0 MB/s) - ‘contractions.py.4’ saved [3110/3110]



In [ ]:

clear_output()

# Trial 1

In [ ]:
train_dataset = pd.read_csv('/content/drive/MyDrive/TA/A1/train.csv')
test_dataset = pd.read_csv('/content/drive/MyDrive/TA/A1/test.csv')

train_dataset['label'] = [1 if x=="positive" else 0 for x in train_dataset['sentiment'] ]
test_dataset['label'] = [1 if x=="positive" else 0 for x in test_dataset['sentiment'] ]

train_dataset = train_dataset.drop('sentiment', axis=1)
test_dataset = test_dataset.drop('sentiment', axis=1)

train_texts, train_labels = train_dataset['review'], train_dataset['label']
test_texts, test_labels = test_dataset['review'], test_dataset['label']

train_texts.reset_index(drop=True, inplace=True)
test_texts.reset_index(drop=True, inplace=True)

from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

train_texts.reset_index(drop=True, inplace=True)
val_texts.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
val_labels.reset_index(drop=True, inplace=True)

from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

import torch

class IMDBdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

        # Ensure all encodings and labels have consistent lengths
        assert all(len(val) == len(self.labels) for val in self.encodings.values()), "Encodings and labels lengths do not match."

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = IMDBdataset(train_encodings, train_labels)
test_dataset = IMDBdataset(test_encodings,test_labels)
val_dataset = IMDBdataset(val_encodings, val_labels)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model = model.to(device=device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
raw_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
raw_model = raw_model.to(device=device)
raw_model.eval()


# Store predictions and actual labels
raw_predictions = []
raw_actuals = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # Move tensors to the same device as the model
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)  # Only needed if you're also evaluating performance

        outputs = raw_model(input_ids, attention_mask=attention_mask)

        # Assuming you're doing classification and want the highest probability class
        logits = outputs[0]
        raw_predicted_labels = torch.argmax(logits, dim=1)
        raw_predictions.extend(raw_predicted_labels.cpu().numpy())

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 20000/20000 [05:51<00:00, 56.84it/s]


In [ ]:
raw_actual_labels = []
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        raw_labels = batch['labels'].to(device)  # Assuming labels are on the same device
        raw_actual_labels.extend(raw_labels.cpu().numpy())

100%|██████████| 20000/20000 [00:07<00:00, 2549.09it/s]


In [ ]:
raw_correct_predictions = sum(p == a for p, a in zip(raw_predictions, raw_actual_labels))
raw_accuracy = raw_correct_predictions / len(raw_predictions)
print(f"Accuracy: {raw_accuracy * 100:.2f}%")

Accuracy: 53.46%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
# Ensure predictions and actual_labels are numpy arrays or compatible formats
precision = precision_score(raw_actual_labels, raw_predictions)
recall = recall_score(raw_actual_labels, raw_predictions)
f1 = f1_score(raw_actual_labels, raw_predictions)
conf_matrix = confusion_matrix(raw_actual_labels, raw_predictions)
mcc = matthews_corrcoef(raw_actual_labels, raw_predictions)

# ROC-AUC score requires probability scores of the positive class, which might need model.predict_proba() or equivalent
# If your model outputs probabilities, you can use:
# roc_auc = roc_auc_score(actual_labels, prediction_probabilities)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Matthews Correlation Coefficient: {mcc:.2f}")
# print(f"ROC-AUC Score: {roc_auc:.2f}")  # Uncomment if you have probability predictions


Precision: 0.57
Recall: 0.31
F1 Score: 0.40
Confusion Matrix:
[[7606 2329]
 [6978 3087]]
Matthews Correlation Coefficient: 0.08


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)

# Fit and transform the training data to create TF-IDF vectors
X_train_tfidf = tfidf_vectorizer.fit_transform(train_texts)

# Transform the test data to TF-IDF vectors
X_test_tfidf = tfidf_vectorizer.transform(test_texts)

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, matthews_corrcoef

# Initialize and train the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, train_labels)

# Predict on the test set
nb_predictions = nb_classifier.predict(X_test_tfidf)

# Compute metrics
nb_accuracy = accuracy_score(test_labels, nb_predictions)
nb_precision = precision_score(test_labels, nb_predictions, average='binary')
nb_recall = recall_score(test_labels, nb_predictions, average='binary')
nb_f1 = f1_score(test_labels, nb_predictions, average='binary')
nb_conf_matrix = confusion_matrix(test_labels, nb_predictions)
nb_mcc = matthews_corrcoef(test_labels, nb_predictions)

print("Naive Bayes Metrics:")
print(f"Accuracy: {nb_accuracy:.4f}")
print(f"Precision: {nb_precision:.4f}")
print(f"Recall: {nb_recall:.4f}")
print(f"F1 Score: {nb_f1:.4f}")
print(f"Confusion Matrix: \n{nb_conf_matrix}")
print(f"Matthews Correlation Coefficient: {nb_mcc:.4f}")


Naive Bayes Metrics:
Accuracy: 0.8612
Precision: 0.8674
Recall: 0.8549
F1 Score: 0.8611
Confusion Matrix: 
[[8620 1315]
 [1460 8605]]
Matthews Correlation Coefficient: 0.7226


# Trial 2

In [ ]:
from bs4 import BeautifulSoup

train_dataset = pd.read_csv('/content/drive/MyDrive/TA/A1/train.csv')
test_dataset = pd.read_csv('/content/drive/MyDrive/TA/A1/test.csv')

train_dataset['label'] = [1 if x=="positive" else 0 for x in train_dataset['sentiment'] ]
test_dataset['label'] = [1 if x=="positive" else 0 for x in test_dataset['sentiment'] ]

train_dataset = train_dataset.drop('sentiment', axis=1)
test_dataset = test_dataset.drop('sentiment', axis=1)

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

test_dataset['review'] = test_dataset['review'].apply(strip_html_tags)
train_dataset['review'] = train_dataset['review'].apply(strip_html_tags)

train_texts, train_labels = train_dataset['review'], train_dataset['label']
test_texts, test_labels = test_dataset['review'], test_dataset['label']

train_texts.reset_index(drop=True, inplace=True)
test_texts.reset_index(drop=True, inplace=True)

from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

train_texts.reset_index(drop=True, inplace=True)
val_texts.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
val_labels.reset_index(drop=True, inplace=True)

from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

import torch

class IMDBdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

        # Ensure all encodings and labels have consistent lengths
        assert all(len(val) == len(self.labels) for val in self.encodings.values()), "Encodings and labels lengths do not match."

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = IMDBdataset(train_encodings, train_labels)
test_dataset = IMDBdataset(test_encodings,test_labels)
val_dataset = IMDBdataset(val_encodings, val_labels)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model = model.to(device=device)

<ipython-input-15-ccb7caa29d84>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
raw_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
raw_model = raw_model.to(device=device)
raw_model.eval()


# Store predictions and actual labels
raw_predictions = []
raw_actuals = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # Move tensors to the same device as the model
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)  # Only needed if you're also evaluating performance

        outputs = raw_model(input_ids, attention_mask=attention_mask)

        # Assuming you're doing classification and want the highest probability class
        logits = outputs[0]
        raw_predicted_labels = torch.argmax(logits, dim=1)
        raw_predictions.extend(raw_predicted_labels.cpu().numpy())

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 20000/20000 [06:00<00:00, 55.49it/s]


In [ ]:
raw_actual_labels = []
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        raw_labels = batch['labels'].to(device)  # Assuming labels are on the same device
        raw_actual_labels.extend(raw_labels.cpu().numpy())

100%|██████████| 20000/20000 [00:07<00:00, 2522.07it/s]


In [ ]:
raw_correct_predictions = sum(p == a for p, a in zip(raw_predictions, raw_actual_labels))
raw_accuracy = raw_correct_predictions / len(raw_predictions)
print(f"Accuracy: {raw_accuracy * 100:.2f}%")

Accuracy: 50.01%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
# Ensure predictions and actual_labels are numpy arrays or compatible formats
precision = precision_score(raw_actual_labels, raw_predictions)
recall = recall_score(raw_actual_labels, raw_predictions)
f1 = f1_score(raw_actual_labels, raw_predictions)
conf_matrix = confusion_matrix(raw_actual_labels, raw_predictions)
mcc = matthews_corrcoef(raw_actual_labels, raw_predictions)

# ROC-AUC score requires probability scores of the positive class, which might need model.predict_proba() or equivalent
# If your model outputs probabilities, you can use:
# roc_auc = roc_auc_score(actual_labels, prediction_probabilities)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Matthews Correlation Coefficient: {mcc:.2f}")
# print(f"ROC-AUC Score: {roc_auc:.2f}")  # Uncomment if you have probability predictions


Precision: 0.60
Recall: 0.02
F1 Score: 0.04
Confusion Matrix:
[[9801  134]
 [9865  200]]
Matthews Correlation Coefficient: 0.02


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)

# Fit and transform the training data to create TF-IDF vectors
X_train_tfidf = tfidf_vectorizer.fit_transform(train_texts)

# Transform the test data to TF-IDF vectors
X_test_tfidf = tfidf_vectorizer.transform(test_texts)

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, matthews_corrcoef

# Initialize and train the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, train_labels)

# Predict on the test set
nb_predictions = nb_classifier.predict(X_test_tfidf)

# Compute metrics
nb_accuracy = accuracy_score(test_labels, nb_predictions)
nb_precision = precision_score(test_labels, nb_predictions, average='binary')
nb_recall = recall_score(test_labels, nb_predictions, average='binary')
nb_f1 = f1_score(test_labels, nb_predictions, average='binary')
nb_conf_matrix = confusion_matrix(test_labels, nb_predictions)
nb_mcc = matthews_corrcoef(test_labels, nb_predictions)

print("Naive Bayes Metrics:")
print(f"Accuracy: {nb_accuracy:.4f}")
print(f"Precision: {nb_precision:.4f}")
print(f"Recall: {nb_recall:.4f}")
print(f"F1 Score: {nb_f1:.4f}")
print(f"Confusion Matrix: \n{nb_conf_matrix}")
print(f"Matthews Correlation Coefficient: {nb_mcc:.4f}")


Naive Bayes Metrics:
Accuracy: 0.8599
Precision: 0.8622
Recall: 0.8590
F1 Score: 0.8606
Confusion Matrix: 
[[8553 1382]
 [1419 8646]]
Matthews Correlation Coefficient: 0.7199


# Trial 3 - Removing accented characters

In [ ]:
from bs4 import BeautifulSoup

train_dataset = pd.read_csv('/content/drive/MyDrive/TA/A1/train.csv')
test_dataset = pd.read_csv('/content/drive/MyDrive/TA/A1/test.csv')

train_dataset['label'] = [1 if x=="positive" else 0 for x in train_dataset['sentiment'] ]
test_dataset['label'] = [1 if x=="positive" else 0 for x in test_dataset['sentiment'] ]

train_dataset = train_dataset.drop('sentiment', axis=1)
test_dataset = test_dataset.drop('sentiment', axis=1)

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

test_dataset['review'] = test_dataset['review'].apply(remove_accented_chars)
train_dataset['review'] = train_dataset['review'].apply(remove_accented_chars)

train_texts, train_labels = train_dataset['review'], train_dataset['label']
test_texts, test_labels = test_dataset['review'], test_dataset['label']

train_texts.reset_index(drop=True, inplace=True)
test_texts.reset_index(drop=True, inplace=True)

from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

train_texts.reset_index(drop=True, inplace=True)
val_texts.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
val_labels.reset_index(drop=True, inplace=True)

from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

import torch

class IMDBdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

        # Ensure all encodings and labels have consistent lengths
        assert all(len(val) == len(self.labels) for val in self.encodings.values()), "Encodings and labels lengths do not match."

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = IMDBdataset(train_encodings, train_labels)
test_dataset = IMDBdataset(test_encodings,test_labels)
val_dataset = IMDBdataset(val_encodings, val_labels)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model = model.to(device=device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
raw_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
raw_model = raw_model.to(device=device)
raw_model.eval()


# Store predictions and actual labels
raw_predictions = []
raw_actuals = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # Move tensors to the same device as the model
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)  # Only needed if you're also evaluating performance

        outputs = raw_model(input_ids, attention_mask=attention_mask)

        # Assuming you're doing classification and want the highest probability class
        logits = outputs[0]
        raw_predicted_labels = torch.argmax(logits, dim=1)
        raw_predictions.extend(raw_predicted_labels.cpu().numpy())

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 20000/20000 [06:04<00:00, 54.93it/s]


In [ ]:
raw_actual_labels = []
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        raw_labels = batch['labels'].to(device)  # Assuming labels are on the same device
        raw_actual_labels.extend(raw_labels.cpu().numpy())

100%|██████████| 20000/20000 [00:06<00:00, 2860.64it/s]


In [ ]:
raw_correct_predictions = sum(p == a for p, a in zip(raw_predictions, raw_actual_labels))
raw_accuracy = raw_correct_predictions / len(raw_predictions)
print(f"Accuracy: {raw_accuracy * 100:.2f}%")

Accuracy: 49.84%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
# Ensure predictions and actual_labels are numpy arrays or compatible formats
precision = precision_score(raw_actual_labels, raw_predictions)
recall = recall_score(raw_actual_labels, raw_predictions)
f1 = f1_score(raw_actual_labels, raw_predictions)
conf_matrix = confusion_matrix(raw_actual_labels, raw_predictions)
mcc = matthews_corrcoef(raw_actual_labels, raw_predictions)

# ROC-AUC score requires probability scores of the positive class, which might need model.predict_proba() or equivalent
# If your model outputs probabilities, you can use:
# roc_auc = roc_auc_score(actual_labels, prediction_probabilities)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Matthews Correlation Coefficient: {mcc:.2f}")
# print(f"ROC-AUC Score: {roc_auc:.2f}")  # Uncomment if you have probability predictions


Precision: 0.50
Recall: 0.98
F1 Score: 0.66
Confusion Matrix:
[[  60 9875]
 [ 157 9908]]
Matthews Correlation Coefficient: -0.05


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)

# Fit and transform the training data to create TF-IDF vectors
X_train_tfidf = tfidf_vectorizer.fit_transform(train_texts)

# Transform the test data to TF-IDF vectors
X_test_tfidf = tfidf_vectorizer.transform(test_texts)

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, matthews_corrcoef

# Initialize and train the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, train_labels)

# Predict on the test set
nb_predictions = nb_classifier.predict(X_test_tfidf)

# Compute metrics
nb_accuracy = accuracy_score(test_labels, nb_predictions)
nb_precision = precision_score(test_labels, nb_predictions, average='binary')
nb_recall = recall_score(test_labels, nb_predictions, average='binary')
nb_f1 = f1_score(test_labels, nb_predictions, average='binary')
nb_conf_matrix = confusion_matrix(test_labels, nb_predictions)
nb_mcc = matthews_corrcoef(test_labels, nb_predictions)

print("Naive Bayes Metrics:")
print(f"Accuracy: {nb_accuracy:.4f}")
print(f"Precision: {nb_precision:.4f}")
print(f"Recall: {nb_recall:.4f}")
print(f"F1 Score: {nb_f1:.4f}")
print(f"Confusion Matrix: \n{nb_conf_matrix}")
print(f"Matthews Correlation Coefficient: {nb_mcc:.4f}")


Naive Bayes Metrics:
Accuracy: 0.8603
Precision: 0.8611
Recall: 0.8613
F1 Score: 0.8612
Confusion Matrix: 
[[8537 1398]
 [1396 8669]]
Matthews Correlation Coefficient: 0.7206


# Trial 4

In [ ]:
from bs4 import BeautifulSoup

train_dataset = pd.read_csv('/content/drive/MyDrive/TA/A1/train.csv')
test_dataset = pd.read_csv('/content/drive/MyDrive/TA/A1/test.csv')

train_dataset['label'] = [1 if x=="positive" else 0 for x in train_dataset['sentiment'] ]
test_dataset['label'] = [1 if x=="positive" else 0 for x in test_dataset['sentiment'] ]

train_dataset = train_dataset.drop('sentiment', axis=1)
test_dataset = test_dataset.drop('sentiment', axis=1)

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

test_dataset['review'] = test_dataset['review'].apply(remove_accented_chars)
train_dataset['review'] = train_dataset['review'].apply(remove_accented_chars)

train_texts, train_labels = train_dataset['review'], train_dataset['label']
test_texts, test_labels = test_dataset['review'], test_dataset['label']

train_texts.reset_index(drop=True, inplace=True)
test_texts.reset_index(drop=True, inplace=True)

from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

train_texts.reset_index(drop=True, inplace=True)
val_texts.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
val_labels.reset_index(drop=True, inplace=True)

from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

import torch

class IMDBdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

        # Ensure all encodings and labels have consistent lengths
        assert all(len(val) == len(self.labels) for val in self.encodings.values()), "Encodings and labels lengths do not match."

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = IMDBdataset(train_encodings, train_labels)
test_dataset = IMDBdataset(test_encodings,test_labels)
val_dataset = IMDBdataset(val_encodings, val_labels)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model = model.to(device=device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
raw_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
raw_model = raw_model.to(device=device)
raw_model.eval()


# Store predictions and actual labels
raw_predictions = []
raw_actuals = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # Move tensors to the same device as the model
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)  # Only needed if you're also evaluating performance

        outputs = raw_model(input_ids, attention_mask=attention_mask)

        # Assuming you're doing classification and want the highest probability class
        logits = outputs[0]
        raw_predicted_labels = torch.argmax(logits, dim=1)
        raw_predictions.extend(raw_predicted_labels.cpu().numpy())

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 20000/20000 [06:04<00:00, 54.93it/s]


In [ ]:
raw_actual_labels = []
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        raw_labels = batch['labels'].to(device)  # Assuming labels are on the same device
        raw_actual_labels.extend(raw_labels.cpu().numpy())

100%|██████████| 20000/20000 [00:06<00:00, 2860.64it/s]


In [ ]:
raw_correct_predictions = sum(p == a for p, a in zip(raw_predictions, raw_actual_labels))
raw_accuracy = raw_correct_predictions / len(raw_predictions)
print(f"Accuracy: {raw_accuracy * 100:.2f}%")

Accuracy: 49.84%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef
# Ensure predictions and actual_labels are numpy arrays or compatible formats
precision = precision_score(raw_actual_labels, raw_predictions)
recall = recall_score(raw_actual_labels, raw_predictions)
f1 = f1_score(raw_actual_labels, raw_predictions)
conf_matrix = confusion_matrix(raw_actual_labels, raw_predictions)
mcc = matthews_corrcoef(raw_actual_labels, raw_predictions)

# ROC-AUC score requires probability scores of the positive class, which might need model.predict_proba() or equivalent
# If your model outputs probabilities, you can use:
# roc_auc = roc_auc_score(actual_labels, prediction_probabilities)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Matthews Correlation Coefficient: {mcc:.2f}")
# print(f"ROC-AUC Score: {roc_auc:.2f}")  # Uncomment if you have probability predictions


Precision: 0.50
Recall: 0.98
F1 Score: 0.66
Confusion Matrix:
[[  60 9875]
 [ 157 9908]]
Matthews Correlation Coefficient: -0.05


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)

# Fit and transform the training data to create TF-IDF vectors
X_train_tfidf = tfidf_vectorizer.fit_transform(train_texts)

# Transform the test data to TF-IDF vectors
X_test_tfidf = tfidf_vectorizer.transform(test_texts)

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, matthews_corrcoef

# Initialize and train the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, train_labels)

# Predict on the test set
nb_predictions = nb_classifier.predict(X_test_tfidf)

# Compute metrics
nb_accuracy = accuracy_score(test_labels, nb_predictions)
nb_precision = precision_score(test_labels, nb_predictions, average='binary')
nb_recall = recall_score(test_labels, nb_predictions, average='binary')
nb_f1 = f1_score(test_labels, nb_predictions, average='binary')
nb_conf_matrix = confusion_matrix(test_labels, nb_predictions)
nb_mcc = matthews_corrcoef(test_labels, nb_predictions)

print("Naive Bayes Metrics:")
print(f"Accuracy: {nb_accuracy:.4f}")
print(f"Precision: {nb_precision:.4f}")
print(f"Recall: {nb_recall:.4f}")
print(f"F1 Score: {nb_f1:.4f}")
print(f"Confusion Matrix: \n{nb_conf_matrix}")
print(f"Matthews Correlation Coefficient: {nb_mcc:.4f}")


Naive Bayes Metrics:
Accuracy: 0.8603
Precision: 0.8611
Recall: 0.8613
F1 Score: 0.8612
Confusion Matrix: 
[[8537 1398]
 [1396 8669]]
Matthews Correlation Coefficient: 0.7206
